In [105]:
from cleaning import database_cleaner
from nlp_pipeline import feature_matrix

import string
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import log_loss, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy  import linkage, dendrogram
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

In [90]:
tamu_df = database_cleaner('../data/tamu_database.json')
tamu_df.head()

,faculty_name,email,google_scholar_link,office,page,phone,faculty_title,paper_titles,abstracts,research_areas
0,A. Daniel Hill,danhill@tamu.edu,https://scholar.google.com/citations?user=EBnW...,RICH 1012,https://engineering.tamu.edu/petroleum/profile...,979-845-2244,Professor,Mechanism of wormholing and its optimal condi...,Acid stimulation is commonly used in carbonat...,Dr. Hill has five patents in oil recovery and ...
1,A. Rashid Hasan,rhasan@tamu.edu,https://scholar.google.com/citations?user=6lMX...,RICH 501E,https://engineering.tamu.edu/petroleum/profile...,979.847.8564,Professor,,,Wellbore Heat transferSystematic modeling of h...
2,Akhil Datta-Gupta,datta-gupta@tamu.edu,https://scholar.google.com/citations?user=Al-S...,RICH 401G,https://engineering.tamu.edu/petroleum/profile...,979-847-9030,University Distinguished Professor,Radius of Investigation and its Generalizatio...,The concept of radius of investigation is fun...,Dr. Datta-Gupta has research interests in rapi...
3,Albertus Retnanto,albertus.retnanto@qatar.tamu.edu,https://scholar.google.com/citations?user=kN7P...,204K,https://engineering.tamu.edu/petroleum/profile...,974-4423-0281,Associate Professor of the Practice,After-Closure Idiosyncrasies of Fracture C...,"Fracture Calibration Tests (FCT), are stra...",Field development and planning Production enha...
4,Aziz Rahman,aziz.rahman@qatar.tamu.edu,https://scholar.google.com/citations?user=PYRt...,204E,https://engineering.tamu.edu/petroleum/profile...,974-4423-0601,Assistant Professor,,,Flow assurance Multiphase pipe flow Wellbore h...


In [91]:
# For nlp, only retaining faculty_name, research_areas, paper_titles, abstracts
df = tamu_df[['faculty_name', 'research_areas', 'paper_titles', 'abstracts']]
df.head()

,faculty_name,research_areas,paper_titles,abstracts
0,A. Daniel Hill,Dr. Hill has five patents in oil recovery and ...,Mechanism of wormholing and its optimal condi...,Acid stimulation is commonly used in carbonat...
1,A. Rashid Hasan,Wellbore Heat transferSystematic modeling of h...,,
2,Akhil Datta-Gupta,Dr. Datta-Gupta has research interests in rapi...,Radius of Investigation and its Generalizatio...,The concept of radius of investigation is fun...
3,Albertus Retnanto,Field development and planning Production enha...,After-Closure Idiosyncrasies of Fracture C...,"Fracture Calibration Tests (FCT), are stra..."
4,Aziz Rahman,Flow assurance Multiphase pipe flow Wellbore h...,,


In [92]:
missing = df['paper_titles'] == ''
sum(missing)

16

In [93]:
# Working with non-missing entries i.e. 26 faculties
df_nlp = df[~missing]
len(df_nlp)

26

In [94]:
df_nlp.head()

,faculty_name,research_areas,paper_titles,abstracts
0,A. Daniel Hill,Dr. Hill has five patents in oil recovery and ...,Mechanism of wormholing and its optimal condi...,Acid stimulation is commonly used in carbonat...
2,Akhil Datta-Gupta,Dr. Datta-Gupta has research interests in rapi...,Radius of Investigation and its Generalizatio...,The concept of radius of investigation is fun...
3,Albertus Retnanto,Field development and planning Production enha...,After-Closure Idiosyncrasies of Fracture C...,"Fracture Calibration Tests (FCT), are stra..."
5,Berna Hascakir,Heavy oil and oil shale recovery with enhanced...,Water and aromatics fraction interaction at e...,Performance predictions of the In-Situ Combus...
6,David Schechter,Spraberry Trend Area Geological and petrophysi...,Gas Injection for EOR in Organic Rich Shales....,Laboratory experiments of gas injection in or...


# Start of NLP pipeline

In [106]:
corpus = df_nlp['paper_titles'].values

In [102]:
def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

def feature_matrix(corpus, tf_idf=True, stem_lem=None, ngram_range=(1,1), max_df=1.0, min_df=1, max_features=None):
    '''
    Returns tf-idf vector for a given corpus (list/array of documents) by default
    Count vector(bag of words) returned when tf_idf is set to False
    Option to include stemming or lemmatizing
    '''
    tokens = [word_tokenize(doc) for doc in corpus]
    tokens_lower = [[word.lower() for word in sent] for sent in tokens]
    tokens_filtered = list(map(filter_tokens, tokens_lower))

    # Stemming-Lemmatizing
    if stem_lem == 'stem':
        stemmer_porter = PorterStemmer()
        tokens_filtered = [' '. join(list(map(stemmer_porter.stem, sent))) for sent in tokens_filtered]


    elif stem_lem == 'lem':
        lemmatizer = WordNetLemmatizer()
        tokens_filtered = [' '.join(list(map(lemmatizer.lemmatize, sent))) for sent in tokens_filtered]

    else:
        tokens_filtered = [' '.join(sent) for sent in tokens_filtered]

    # Vectorizing
    if tf_idf:
        vectorizer = TfidfVectorizer(stop_words=stopwords_, ngram_range=ngram_range, 
                                     max_df=max_df, min_df=min_df, max_features=max_features)
        matrix = vectorizer.fit_transform(tokens_filtered) # sparse matrix

    else:
        vectorizer = CountVectorizer(stop_words=stopwords_, ngram_range=ngram_range, 
                                     max_df=max_df, min_df=min_df, max_features=max_features)
        matrix = vectorizer.fit_transform(tokens_filtered) # sparse matrix

    return vectorizer, matrix

In [107]:
vectorizer, matrix = feature_matrix(corpus, tf_idf=True, stem_lem=None, ngram_range=(1,1), 
                                    max_df=1.0, min_df=1, max_features=None)

In [108]:
matrix.shape

(26, 1690)